<a href="https://colab.research.google.com/github/ryanskytree/Machine-Learning-Practice/blob/main/Compare_k_ppr_heat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch_geometric
import sys
sys.path.append("/content/drive/MyDrive/KorderGraphDiffusionConv/")
import GDC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.7 MB/s eta 0:00:00


In [ ]:
import argparse
import os.path as osp
import time

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.logging import init_wandb, log
from torch_geometric.nn import GCNConv
import numpy as np

dataset='Cora'
lr=0.01
hidden_channels=16
epochs=1000
use_gdc='Use GDC'
wandb='Track experiment'


if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

init_wandb(
    name=f'GCN-{dataset}',
    lr=lr,
    epochs=epochs,
    hidden_channels=hidden_channels,
    device=device,
)

dataset = Planetoid(root='/tmp/cora', name='Cora', transform=T.NormalizeFeatures())


Processing...
Done!


In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_weight=None):
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_attr)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_attr).argmax(dim=-1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs

def getacc():
    best_val_acc = test_acc = 0
    times = []
    for epoch in range(1, epochs + 1):
        loss = train()
        train_acc, val_acc, tmp_test_acc = test()
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
    return test_acc


In [ ]:
Pure_GCN_test=[]
for transorm_kernel in ['korder','none','ppr','heat']:
    transorm_list = []
    data = dataset[0].to(device)
    if transorm_kernel == 'none':
        pass
    else:
        transform = GDC.GDC(
            self_loop_weight=1,
            normalization_in='sym',
            normalization_out='col',
            diffusion_kwargs=dict(method = transorm_kernel, alpha=0.05,t=3),
            sparsification_kwargs=dict(method='topk', k=128, dim=0),
            exact=True,
            kinput=18
        )
        data = transform(data)
    for gcn_test_count in range(1):

        model = GCN(
          in_channels=dataset.num_features,
          hidden_channels=hidden_channels,
          out_channels=dataset.num_classes,
        ).to(device)
        optimizer = torch.optim.Adam([
            dict(params=model.conv1.parameters(), weight_decay=5e-4),
            dict(params=model.conv2.parameters(), weight_decay=5e-5)
        ], lr)  # Only perform weight-decay on first convolution.

        transorm_list.append(getacc())
    Pure_GCN_test.append(transorm_list)
print('k',np.average(Pure_GCN_test[0]))
print(np.average(Pure_GCN_test[1]))
print(np.average(Pure_GCN_test[2]))
print(np.average(Pure_GCN_test[3]))